In [ ]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import numpy as np
from enterprise.signals.parameter import Parameter
import matplotlib.pyplot as plt

In [ ]:
import george
import gaussproc
from gaussproc import *
import cPickle as pickle

In [ ]:
pickle_path = '../enterprise/datafiles/starsecc_gp_13x14nodes_30yr.pkl'
gppkl = pickle.load( open( pickle_path, "rb" ) )

# Set george kernel parameters to previously-trained MAP
# Compute factorisation of kernel based on sampled points
gp = []
for ii in range(len(gppkl)):
    gp_kparams = np.exp(gppkl[ii].kernel_map)
    gp.append( george.GP( gp_kparams[0] * \
                          george.kernels.ExpSquaredKernel(gp_kparams[1:],ndim=len(gp_kparams[1:])) ) )
    gp[ii].compute(gppkl[ii].x, gppkl[ii].yerr)
    gwb_popparam_ndims = len(gp_kparams[1:])

gwb_popparam = pickle_path.split('/')[-1].split('_')
for word in gwb_popparam:
    if word in ['stars','ecc','gas','starsecc']:
        gwb_popparam = word
        break

In [ ]:
class PSD(object):
    
    # This is a dict containing all defined PSD instances,
    # keyed on the name of the pdf type, e.g. powerlaw.
    _registry = {}
    
    def __new__(cls, name, *args, **kwargs):
        # Generates a new PSD object instance, and adds it
        # it the registry, using name as the key.  Name must be unique,
        # a new instance with a given name will over-write the existing
        # one.
        psd = super(PSD, cls).__new__(cls, name, *args, **kwargs)
        cls._register(psd, name)
        return psd
    
    def __init__(self, name):
        self.params = []
    
    @classmethod
    def _register(cls, psd, name):
        """Add a psd to the registry using the specified name
        (which will be converted to lower case).  If an existing psd
        of the same name exists, it will be replaced with the new one.  
        The PSD instance's name attribute will be updated for
        consistency."""
        cls._registry[name.lower()] = psd
        psd._name = name.lower()
        
    @classmethod
    def names(cls):
        return cls._registry.keys()
        
    @property
    def name(self): return self._name

    @classmethod
    def get(cls, name):
       
        # Be case-insensitive
        name = name.lower()
        # First see if name matches
        if name in cls._registry.keys(): 
            return cls._registry[name]
        # Nothing matched, raise an error
        raise KeyError("PSD name '%s' is not defined" % name)
        
    def add_param(self, param):
        setattr(self, param.name, param)
        self.params += [param]
        
    def get_params(self):
        return np.array([param.value for param in self.params])
        
    def get_psd(self, f):
        """Must be defined in derived class"""
        raise NotImplementedError

In [ ]:
class PowerLawSpectrum(PSD):
        
    def __init__(self, name):
        super(PowerLawSpectrum, self).__init__(name)
    
        self.add_param(Parameter('A', -15.0, uncertainty=0.1,
                                 description='Red Noise Amplitude [log-scale]'))
        self.add_param(Parameter('gamma', 4.33, uncertainty=0.1,
                                 description='Red Noise Spectral index'))
    
    def get_psd(self, f):
        df = f[0]
        f1yr = 1 / 3.16e7
        Amp = 10**self.A.value
        gamma = self.gamma.value
        psd = Amp ** 2 / 12 / np.pi ** 2 * f1yr ** (gamma - 3) * f ** (-gamma) * df
        return psd

In [ ]:
class TurnoverSpectrum(PSD):
        
    def __init__(self, name):
        super(TurnoverSpectrum, self).__init__(name)
    
        self.add_param(Parameter('A', -15.0, uncertainty=0.1,
                                 description='Red Noise Amplitude [log-scale]'))
        self.add_param(Parameter('gamma', 4.33, uncertainty=0.1,
                                 description='Red Noise Spectral index'))
        self.add_param(Parameter('fbend', 1e-8, uncertainty=0.1,
                                 description='Spectral bend frequency'))
        self.add_param(Parameter('kappaturn', 7.0/3.0, uncertainty=0.1,
                                 description='Slope of low-frequency spectrum'))
    
    def get_psd(self, f):
        df = f[0]
        f1yr = 1 / 3.16e7
        Amp = 10**self.A.value
        gamma = self.gamma.value
        fbend = self.fbend.value
        kappaturn = self.kappaturn.value
        psd = ( Amp ** 2 / 12 / np.pi ** 2 * f1yr ** (gamma - 3) * f ** (-gamma) * df 
               / ( 1.0 + (fbend / f) ** kappaturn) )
        return psd

In [ ]:
class GPSpectrum(PSD):
        
    def __init__(self, name, nf):
        super(GPSpectrum, self).__init__(name)
        self.nf = nf
    
        for ii in range(nf):
            self.add_param(Parameter('psd_white_'+str(ii), 0.0, uncertainty=0.1,
                                     description='Zero-mean unit-variance Gaussian draw'))
        self.add_param(Parameter('A', -15.0, uncertainty=0.1,
                                description='Red Noise Amplitude [log-scale]'))
        self.add_param(Parameter('rho_stars', 4.0, uncertainty=0.1,
                                 description='Mass density of stars in galactic center [Log-scale]'))
        self.add_param(Parameter('ecc', 0.5, uncertainty=0.1,
                                 description='Initial eccentricity of SMBHBs'))
    
    def get_psd(self, f, gp):
        df = f[0]
        f1yr = 1 / 3.16e7
        psd_white = self.get_params()[:self.nf]
        Amp = 10**self.A.value
        rho_stars = self.rho_stars.value
        ecc = self.ecc.value
        env_param = np.array([rho_stars, ecc])
        
        rho_pred = np.zeros((len(f),2))
        for ii in range(len(f)):
            mu_pred, cov_pred = gp[ii].predict(gppkl[ii].y, [env_param])
            if np.diag(cov_pred) < 0.0:
                rho_pred[ii,0], rho_pred[ii,1] = mu_pred, 1e-5 * mu_pred
            else:
                rho_pred[ii,0], rho_pred[ii,1] = mu_pred, np.sqrt(np.diag(cov_pred))

        # transforming from zero-mean unit-variance variable to rho
        psd = ( 2.0*np.log10(Amp) - np.log10(12.0 * np.pi ** 2 * f ** 3) + 
               np.log10(df) + psd_white*rho_pred[:,1] + rho_pred[:,0] )
        psd = 10.0**psd
        return psd

In [ ]:
# call this to register (could go in an __init__.py)      
PowerLawSpectrum('powerlaw')

In [ ]:
# call this to register (could go in an __init__.py)      
TurnoverSpectrum('turnover')

In [ ]:
GPSpectrum('gpspectrum',len(gp))

In [ ]:
# test
psd1 = PSD.get('powerlaw')
psd2 = PSD.get('gpspectrum')

In [ ]:
T = 30 * 3.16e7
f = np.linspace(1/T, psd2.nf/T, psd2.nf)

In [ ]:
# change the value
psd1.A.value = -15.5
psd1.gamma.value = 13./3.

# diffuse stellar environment 
# and zero eccentricity
psd2.A.value = -15.5
psd2.rho_stars.value = 1.0
psd2.ecc.value = 0.0

plt.loglog(f, psd1.get_psd(f))
plt.loglog(f, psd2.get_psd(f, gp))
plt.show()